# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""7214098495505223835""","""../../../../data/SampleHDS.jso…",1.5658e19,"""9125746113062090954""","""13720266205980305223""","""17203863101706440101""",false
"""16925946624136288702""","""../../../../data/SampleHDS.jso…",9.7340e18,"""9122969655173187130""","""13720266205980305223""","""5677219347270663792""",true
"""3253644328268054947""","""../../../../data/SampleHDS.jso…",3.4717e18,"""7963550500859883798""","""8096707801333887523""","""17203863101706440101""",false
"""4215141485855284346""","""../../../../data/SampleHDS.jso…",1.1341e19,"""9122969655173187130""","""8317988125252490892""","""17203863101706440101""",false
"""18068483211249972322""","""../../../../data/SampleHDS.jso…",6.8087e18,"""9125746113062090954""","""8317988125252490892""","""981697134326111858""",true
"""4104324425473679245""","""../../../../data/SampleHDS.jso…",3.7513e18,"""7963550500859883798""","""5340830511425436750""",null,false
"""11702730099285820339""","""../../../../data/SampleHDS.jso…",1.6317e19,"""7963550500859883798""","""8317988125252490892""","""981697134326111858""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""2373592059457764073""","""../../../../data/SampleHDS.jso…",6.9199e18,"""2880600467473429942""","""FirstMortgage30yr""","""6772501305515576314""","""Rejected"""
"""17241819709490878424""","""../../../../data/SampleHDS.jso…",1.5791e19,"""13237482271131638777""","""FirstMortgage30yr""","""17413254661474864988""","""Accepted"""
"""322130473586871040""","""../../../../data/SampleHDS.jso…",6.7790e18,"""8407542876692354426""","""MoneyMarketSavingsAccount""","""6772501305515576314""","""Rejected"""
"""1482037313563932753""","""../../../../data/SampleHDS.jso…",1.4837e18,"""13237482271131638777""","""BasicChecking""","""6772501305515576314""","""Rejected"""
"""9820393012684327919""","""../../../../data/SampleHDS.jso…",1.1879e19,"""2880600467473429942""","""BasicChecking""","""4628437209402000446""","""Accepted"""
"""6200133409996739200""","""../../../../data/SampleHDS.jso…",1.7281e19,"""8407542876692354426""","""UPlusFinPersonal""",null,"""Rejected"""
"""2305326543846589043""","""../../../../data/SampleHDS.jso…",1.5397e19,"""8407542876692354426""","""BasicChecking""","""4628437209402000446""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""10940758635026206683""","""../../../../data/SampleHDS.jso…",3.0582e18,"""10283509424210239021""","""FirstMortgage30yr""","""15274554402532124515""","""Rejected"""
"""13492107503574211872""","""../../../../data/SampleHDS.jso…",1.1463e18,"""4342418725627949777""","""FirstMortgage30yr""","""1926519464317187260""","""Accepted"""
"""7969642468172285795""","""../../../../data/SampleHDS.jso…",7.8023e18,"""7895422772354746500""","""MoneyMarketSavingsAccount""","""15274554402532124515""","""Rejected"""
"""1313107311780088065""","""../../../../data/SampleHDS.jso…",1.5057e19,"""4342418725627949777""","""BasicChecking""","""15274554402532124515""","""Rejected"""
"""9784836628120965613""","""../../../../data/SampleHDS.jso…",9.0925e18,"""10283509424210239021""","""BasicChecking""","""15166405519848142881""","""Accepted"""
"""4957994631410446949""","""../../../../data/SampleHDS.jso…",2.9083e18,"""7895422772354746500""","""UPlusFinPersonal""",null,"""Rejected"""
"""17660250685348595059""","""../../../../data/SampleHDS.jso…",4.4894e18,"""7895422772354746500""","""BasicChecking""","""15166405519848142881""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""12566719704964328590""",5.7964e17,"""7316055550216647036""","""FirstMortgage30yr""","""14953231644066734692""","""Rejected"""
"""3484814720753155743""",7.2890e18,"""17450610036069767591""","""FirstMortgage30yr""","""7571795107142995895""","""Accepted"""
"""6508277315004576810""",1.4319e18,"""1411779264622791539""","""MoneyMarketSavingsAccount""","""14953231644066734692""","""Rejected"""
"""7336094545971923405""",1.7302e19,"""17450610036069767591""","""BasicChecking""","""14953231644066734692""","""Rejected"""
"""15033152049786308567""",1.1405e19,"""7316055550216647036""","""BasicChecking""","""4427937609119853381""","""Accepted"""
"""9482412601445830876""",1.1185e19,"""1411779264622791539""","""UPlusFinPersonal""",null,"""Rejected"""
"""13056348536235662935""",1.2898e19,"""1411779264622791539""","""BasicChecking""","""4427937609119853381""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'